In [1]:
import streamlit as st
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the k-means model
with open(r'C:\Users\shumo\Desktop\Data Science\Shubham\Project\Project343\kmeans_model.pkl', 'rb') as model_file:
    kmeans_model = pickle.load(model_file)

# Load the preprocessed data
with open(r'C:\Users\shumo\Desktop\Data Science\Shubham\Project\Project343\preprocessed_data.pkl', 'rb') as data_file:
    netflix_data = pickle.load(data_file)

# Create a text representation of the combined categorical columns
netflix_data['combined'] = netflix_data[['type', 'title', 'director', 'cast', 'country', 'rating', 'listed_in']].agg(' '.join, axis=1)

# Convert text data to TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(netflix_data['combined'])

# Streamlit app
st.title('Movie Recommendation System')

# Ask user to manually input keywords
selected_keywords = st.text_input('Enter keywords (comma-separated):')

# Ask user to select a range of years
year_range = st.slider('Select a range of years:', min_value=int(netflix_data['release_year'].min()), 
                      max_value=int(netflix_data['release_year'].max()), value=(int(netflix_data['release_year'].min()), int(netflix_data['release_year'].max())))

# Initialize session state only if keywords change
if 'offset' not in st.session_state or st.session_state.keywords != selected_keywords:
    st.session_state.offset = 0
    st.session_state.recommended_movies = set()
    st.session_state.keywords = selected_keywords

recommend_button = st.button('Get Recommendations')

# Check if the keywords contain a valid category in the dataset (case-insensitive)
keywords_list = [keyword.strip().lower() for keyword in selected_keywords.split(',')]
valid_keywords = set(netflix_data['listed_in'].str.lower().unique())
if not set(keywords_list).issubset(valid_keywords):
    st.warning("Invalid keywords. Please enter valid keywords.")
elif recommend_button:
    # Filter movies based on keywords and release year range
    filtered_movies = netflix_data[
        (netflix_data['listed_in'].str.lower().str.contains('|'.join(keywords_list), case=False)) &
        (netflix_data['release_year'].between(year_range[0], year_range[1]))
    ]

    # Get fixed number of recommendations (5) with offset, avoiding repetition
    num_recommendations = 5
    offset_movies = [movie for movie in filtered_movies['title'].tolist()[st.session_state.offset : st.session_state.offset + num_recommendations] if movie not in st.session_state.recommended_movies]
    
    # Append the new recommendations to the old ones
    st.session_state.recommended_movies.update(offset_movies)

    # Display recommendations
    st.subheader('Recommended Movies/TV Shows:')
    for i, movie in enumerate(st.session_state.recommended_movies, start=1):
        st.write(f"{i}. {movie}")

    # Add "Search More" button
    #if st.button('Search More'):
        # Increment offset for the next set of 5 fresh recommendations
        #st.session_state.offset += num_recommendations
        # Get fresh recommendations based on the selected range of "release_year", avoiding repetition
        #fresh_recommendations = [movie for movie in filtered_movies['title'].tolist()[st.session_state.offset : st.session_state.offset + num_recommendations] if movie not in st.session_state.recommended_movies]
        
        # Append the fresh recommendations to the old ones
        #st.session_state.recommended_movies.update(fresh_recommendations)

        # Display additional recommendations
        #st.subheader('More Recommended Movies/TV Shows:')
        #for i, movie in enumerate(fresh_recommendations, start=len(st.session_state.recommended_movies) - num_recommendations + 1):
            #st.write(f"{i}. {movie}")

# Add "Reset" button to clear old recommendations
if st.button('Reset'):
    st.session_state.recommended_movies = set()
    st.session_state.offset = 0


2024-02-21 15:42:10.193 
  command:

    streamlit run C:\Users\shumo\Desktop\Data Science\Shubham\Project\Project343\ipykernel_launcher.py [ARGUMENTS]
2024-02-21 15:42:10.193 Session state does not function when running a script without `streamlit run`
